# Transformer Architecture
Code is Frome [Here](https://nlp.seas.harvard.edu/annotated-transformer/#background)

In [3]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [5]:
!python -m spacy download de_core_news_sm

     ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
     --------------------------------------- 0.0/19.1 MB 660.6 kB/s eta 0:00:29
      --------------------------------------- 0.3/19.1 MB 2.9 MB/s eta 0:00:07
     - -------------------------------------- 0.6/19.1 MB 5.1 MB/s eta 0:00:04
     -- ------------------------------------- 1.1/19.1 MB 6.1 MB/s eta 0:00:03
     --- ------------------------------------ 1.5/19.1 MB 6.7 MB/s eta 0:00:03
     ---- ----------------------------------- 2.0/19.1 MB 7.3 MB/s eta 0:00:03
     ---- ----------------------------------- 2.3/19.1 MB 7.5 MB/s eta 0:00:03
     ----- ---------------------------------- 2.7/19.1 MB 7.6 MB/s eta 0:00:03
     ------ --------------------------------- 3.2/19.1 MB 7.8 MB/s eta 0:00:03
     ------- -------------------------------- 3.7/19.1 MB 8.1 MB/s eta 0:00:02
     -------- ------------------------------- 4.1/19.1 MB 8.2 MB/s eta 0:00:02
     --------- ------------------------------ 4.5/19.1 MB 

DEPRECATION: https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.2.0/de_core_news_sm-3.2.0-py3-none-any.whl#egg=de_core_news_sm==3.2.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617


In [6]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/13.9 MB ? eta -:--:--
     --------------------------------------- 0.0/13.9 MB 640.0 kB/s eta 0:00:22
      --------------------------------------- 0.2/13.9 MB 2.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/13.9 MB 5.4 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/13.9 MB 6.4 MB/s eta 0:00:02
     ---- ----------------------------------- 1.7/13.9 MB 7.2 MB/s eta 0:00:02
     ------ --------------------------------- 2.2/13.9 MB 7.8 MB/s eta 0:00:02
     ------- -------------------------------- 2.7/13.9 MB 8.1 MB/s eta 0:00:02
     -------- ------------------------------- 3.1/13.9 MB 8.2 MB/s eta 0:00:02
     ---------- ----------------------------- 3.6/13.9 MB 8.5 MB/s eta 0:00:02
     ----------- ---------------------------- 4.1/13.9 MB 9.0 MB/s eta 0:00:02
     ------------- -------------------------- 4.6/13.9 MB 9.1 MB/s eta 0:00:02
     -------------- ------------------------- 5.0/13.9 MB 

DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl#egg=en_core_web_sm==3.2.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617


In [4]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None